#### 1. 常用的数据扩增库     
- #### torchvision      
https://github.com/pytorch/vision      
pytorch官方提供的数据扩增库，提供了基本的数据数据扩增方法，可以无缝与torch进行集成；但数据扩增方法种类较少，且速度中等；       
        
- #### imgaug         
https://github.com/aleju/imgaug      
imgaug是常用的第三方数据扩增库，提供了多样的数据扩增方法，且组合起来非常方便，速度较快；      
       
- #### albumentations       
https://albumentations.readthedocs.io     
https://github.com/albumentations-team/albumentations       
是常用的第三方数据扩增库，提供了多样的数据扩增方法，对图像分类、语义分割、物体检测和关键点检测都支持，速度较快。    

In [1]:
import cv2
import os
import numpy as np
import torchvision.transforms as transforms
from common_tools import set_seed, transform_invert
import torch.nn as nn
import torch

from imgaug import augmenters as iaa
from PIL import Image, ImageEnhance
from matplotlib import pyplot as plt

In [98]:
# grayscaling
def grayscaling(img, path):
    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    path = path + '_gray.jpg'
    cv2.imwrite(path, image_gray)

In [99]:
def gaussianblurring(img, path):
    random_range = (0.5,1.5)
    blur = iaa.GaussianBlur(sigma=random_range)
    image_gaublur = blur.augment_image(img)
    path = path + '_gaussianblur.jpg'
    cv2.imwrite(path, image_gaublur)

In [100]:
def cropping(img, path):
    height, width, _ = img.shape
    crop_1 = iaa.CropToFixedSize(int(width*0.8), int(height*0.8), position="left-top")
    #crop_2 = iaa.CropToFixedSize(int(width*0.8), int(height*0.8), position="right-bottom")
    image_crop_1 = cv2.resize(crop_1.augment_image(img), (width, height))
    path = path + '_crop_1.jpg'
    cv2.imwrite(path, image_crop_1)

In [101]:
def add_snow(image):
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS
    image_HLS = np.array(image_HLS, dtype = np.float64)
    brightness_coefficient = 2.5
    snow_point=120 ## increase this for more snow
    image_HLS[:,:,1][image_HLS[:,:,1]<snow_point] = image_HLS[:,:,1][image_HLS[:,:,1]<snow_point]*brightness_coefficient ## scale pixel values up for channel 1(Lightness)
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255
    image_HLS = np.array(image_HLS, dtype = np.uint8)
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB
    return image_RGB

def snowing(img, path):
    image_snow = add_snow(img)
    path = path + '_snow.jpg'
    cv2.imwrite(path, image_snow)

In [102]:
def brightening(img, path):
    # convert cv image to pil image
    pil_image = Image.fromarray(img)
    # enhance pil image by changing the brightness level
    enhancer = ImageEnhance.Brightness(pil_image)
    # original = 1.0; brighter > 1.0
    brightness_level = 1.8
    brighter_image = enhancer.enhance(brightness_level)
    # converting pil image back to cv image
    brighter_image = np.array(brighter_image)
    path = path + '_bright.jpg'
    cv2.imwrite(path, brighter_image)

In [103]:
def adding_fog(img, path):
    fog = iaa.Fog()#seed=88 可以随机固定
    image_fog = fog.augment_image(img)
    path = path + 'fog.jpg'
    cv2.imwrite(path, image_fog)

In [104]:
def adding_clouds(img, path):
    cloud = iaa.CloudLayer(
        intensity_mean=(190, 255), intensity_freq_exponent=(-1.5, -2.0), intensity_coarse_scale=10,
        alpha_min=0, alpha_multiplier=(0.25, 0.75), alpha_size_px_max=(2, 8), alpha_freq_exponent=(-2.5, -2.0),
        sparsity=(0.8, 1.0), density_multiplier=(2.0, 2.5))
    image_cloud = cloud.augment_image(img)
    path = path + '_cloud.jpg'
    cv2.imwrite(path, image_cloud)

In [105]:
#rain

In [106]:
#noise

In [6]:
#tencrop
train_transform = transforms.Compose([    
    transforms.TenCrop(224, vertical_flip=False),
    # crops是 tencrop的一个输出，是tuple形式，长度是10的，每一个元素是一张pil image， 放在了list里面，列表解析式，用totensor 转换张量
    transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops])),
])

In [7]:
img = Image.open(path).convert('RGB')
img = train_transform(img)

In [8]:
img.shape

torch.Size([10, 3, 224, 224])

In [9]:
ncrops, c, h, w = img.shape

In [10]:
type(img)

torch.Tensor

In [11]:
img.max()

tensor(1.)

In [12]:
def tencrop(img, path):
    for n in range(ncrops):
            img_tensor = img[n, ...]  # C H W
            img_n = transform_invert(img_tensor, train_transform)
            
            sppath = os.path.splitext(path)
            path = sppath[0]
            
            path_n = path + '_tencrop_{}.jpg'.format(n)
            img_n.save(path_n, quality=95)

In [125]:
#tencrop_vertical
train_transform_v = transforms.Compose([    
    transforms.TenCrop(224, vertical_flip=True),
    # crops是 tencrop的一个输出，是tuple形式，长度是10的，每一个元素是一张pil image， 放在了list里面，列表解析式，用totensor 转换张量
    transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops])),
])

In [126]:
img = Image.open(path).convert('RGB')
img = train_transform_v(img)
ncrops, c, h, w = img.shape

In [127]:
print(ncrops, c, h, w)

10 3 224 224


In [128]:
def tencrop_v(img, path):
    for n in range(ncrops):
        img_tensor = img[n, ...]  # C H W
        img_n = transform_invert(img_tensor, train_transform_v)
        
        sppath = os.path.splitext(path)
        path = sppath[0]
        
        path_n = path + '_tencrop_v_{}.jpg'.format(n)
        img_n.save(path_n, quality=95)

In [14]:
color_transform = transforms.Compose([
    transforms.ColorJitter(brightness=1, contrast=0.5,saturation=1),
    transforms.ToTensor(),
])

In [15]:
img = Image.open(path).convert('RGB')
img = color_transform(img)

In [16]:
type(img)

torch.Tensor

In [19]:
#colorJitter
def colorJitter(img, path):
    img_cj = transform_invert(img, color_transform)
    path_cj = path + '_colorji.jpg'
    img_cj.save(path_cj, quality=95)

In [77]:
rotation_transform = transforms.Compose([
    transforms.RandomRotation((90), expand=True),
    transforms.ToTensor(),
])

In [92]:
img = Image.open(path).convert('RGB')
img = rotation_transform(img)

In [93]:
#RandomRotation
def randomrotation(img, path):
    img_rr = transform_invert(img, rotation_transform)
    sppath = os.path.splitext(path)
    path_rr = sppath[0] + '_rotation.jpg'
    img_rr.save(path_rr, quality=95)    

In [81]:
path

'E:\\project\\LVPS\\input\\data\\0\\10948653530.jpg'

In [83]:
sppath

('E:\\project\\LVPS\\input\\data\\0\\10948653530', '.jpg')

In [3]:
root_path = "E:\project\LVPS\input\data"
path = os.path.join(root_path, '0')
name = os.listdir(path) #是一个list
path = os.path.join(path,name[0])

img=cv2.imread(path)

In [4]:
path

'E:\\project\\LVPS\\input\\data\\0\\0.jpg'

In [ ]:
grayscaling(img, path)

In [29]:
gaussianblurring(img, path)

In [39]:
cropping(img, path)

In [48]:
snowing(img, path)

In [53]:
brightening(img, path)

In [69]:
adding_fog(img, path)

In [79]:
adding_clouds(img, path)

In [13]:
tencrop(img, path)

In [129]:
tencrop_v(img, path)

AttributeError: 'Tensor' object has no attribute 'astype'

In [20]:
colorJitter(img, path)

In [94]:
randomrotation(img, path)

In [82]:
import torch.nn as nn
import torch

In [85]:
m = nn.ReLU()
input = torch.randn(7)

print("输入处理前图片：")
print(input)

output = m(input)

print("ReLU输出：")
print(output)
print("输出的尺度：")
print(output.size())

print("输入处理后图片：")
print(input)

输入处理前图片：
tensor([ 0.5639,  0.1311,  0.4792, -2.1313, -0.4849,  1.5956, -1.6062])
ReLU输出：
tensor([0.5639, 0.1311, 0.4792, 0.0000, 0.0000, 1.5956, 0.0000])
输出的尺度：
torch.Size([7])
输入处理后图片：
tensor([ 0.5639,  0.1311,  0.4792, -2.1313, -0.4849,  1.5956, -1.6062])
